<a href="https://colab.research.google.com/github/SERVIR/GEDI_Earthhub_ARSET_Training/blob/main/gedi_l1a_convert_to_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install h5py pandas


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/PhD_Main_Folder/Geo_Data/GEDI_DEV_GUAN

/content/drive/MyDrive/PhD_Main_Folder/Geo_Data/GEDI_DEV_GUAN


In [ ]:
import h5py
import pandas as pd

file_path = 'GEDI02_A_2021270173239_O15817_02_T03105_02_003_02_V002.h5'

with h5py.File(file_path, 'r') as f:
    beam_names = [key for key in f.keys() if key.startswith('BEAM')]
    data_frames = []

    for beam in beam_names:
        try:
            # Required datasets
            lat = f[f'{beam}/lat_lowestmode'][:]
            lon = f[f'{beam}/lon_lowestmode'][:]
            shot_num = f[f'{beam}/shot_number'][:]
            rh_all = f[f'{beam}/rh'][:]  # shape: (N, 101)

            # Pull RH98 (the 99th element since Python is 0-indexed)
            rh98 = rh_all[:, 98]

            # Create DataFrame
            df = pd.DataFrame({
                'beam': beam,
                'shot_number': shot_num,
                'latitude': lat,
                'longitude': lon,
                'rh98': rh98
            })

            # Optionally include all RH metrics (uncomment if needed)
            # for i in range(rh_all.shape[1]):
            #     df[f'rh{i}'] = rh_all[:, i]

            data_frames.append(df)

        except Exception as e:
            print(f"Skipping {beam} due to error: {e}")
            continue

# Combine and save
combined_df = pd.concat(data_frames, ignore_index=True)
combined_df.to_csv('GEDI_L2A_spatial_rh98_data.csv', index=False)

print("Saved GEDI spatial and rh98 data to CSV.")


Saved GEDI spatial and rh98 data to CSV.
